<a href="https://colab.research.google.com/github/viktoriya-bel/Tensorflow-labs/blob/main/Exercise_7_sarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Обучение  CNN на полном наборе данных "Определение сарказма"

In [5]:
# нужные импорты
import json
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Сначала надо получить данные.

In [7]:
# https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json - не работает
!wget --no-check-certificate \
    "https://storage.googleapis.com/learning-datasets/sarcasm.json" \
    -O "/tmp/sarcasm.json"

with open("/tmp/sarcasm.json", 'r') as f:
    datastore = json.load(f)


--2023-05-29 13:14:20--  https://storage.googleapis.com/learning-datasets/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.128, 74.125.137.128, 142.250.141.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.09s   

2023-05-29 13:14:20 (60.8 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



In [8]:
# заполнение данными
sentences = [] 
labels = []
urls = []
for item in datastore:
  sentences.append(item['headline'])
  labels.append(item['is_sarcastic'])
  urls.append(item['article_link'])

# инициализация токенайзера
tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
print(word_index)

{'to': 1, 'of': 2, 'the': 3, 'in': 4, 'for': 5, 'a': 6, 'on': 7, 'and': 8, 'with': 9, 'is': 10, 'new': 11, 'trump': 12, 'man': 13, 'from': 14, 'at': 15, 'about': 16, 'you': 17, 'this': 18, 'by': 19, 'after': 20, 'up': 21, 'out': 22, 'be': 23, 'how': 24, 'as': 25, 'it': 26, 'that': 27, 'not': 28, 'are': 29, 'your': 30, 'his': 31, 'what': 32, 'he': 33, 'all': 34, 'just': 35, 'who': 36, 'has': 37, 'will': 38, 'more': 39, 'one': 40, 'into': 41, 'report': 42, 'year': 43, 'why': 44, 'have': 45, 'area': 46, 'over': 47, 'donald': 48, 'u': 49, 'day': 50, 'says': 51, 's': 52, 'can': 53, 'first': 54, 'woman': 55, 'time': 56, 'like': 57, 'her': 58, "trump's": 59, 'old': 60, 'no': 61, 'get': 62, 'off': 63, 'an': 64, 'life': 65, 'people': 66, 'obama': 67, 'now': 68, 'house': 69, 'still': 70, "'": 71, 'women': 72, 'make': 73, 'was': 74, 'than': 75, 'white': 76, 'back': 77, 'my': 78, 'i': 79, 'clinton': 80, 'down': 81, 'if': 82, '5': 83, 'when': 84, 'world': 85, 'could': 86, 'we': 87, 'their': 88, 'be

In [9]:
# создание последовательности
sequences = tokenizer.texts_to_sequences(sentences)
print(sequences)

[[47, 5], [3, 21, 1, 5, 8], [1, 38, 45, 1], [35, 1, 28, 21, 9], [4, 3, 94, 91], [3, 72], [3, 5], [18, 38, 30, 1, 98, 82, 17, 5, 31], [], [59, 5, 3], [13, 36, 4], [7, 8], [12, 8, 1], [37, 5, 6], [4, 2], [5], [44, 1, 23, 19], [19], [1, 35], [], [37], [12, 27, 5, 20, 34, 31], [55, 36, 74, 8], [18, 10, 44, 17, 1, 3], [3, 86, 2], [44, 1, 23], [67, 1], [77], [20], [10, 3, 2, 3], [5, 10], [7, 32, 99, 57, 1], [1, 2, 20], [45, 1], [4], [82, 17, 6, 15, 3], [], [1, 3, 2, 3, 9], [3, 2], [7, 5, 43], [1, 2], [1, 64, 18, 31, 65], [77, 41], [14], [3, 2, 18], [], [27, 72, 39, 5], [1, 65, 4, 71, 1], [68, 10], [19], [], [4, 15, 79, 4], [6, 65], [48, 12, 5, 6, 2, 4, 3], [48, 12, 45, 3, 1], [11], [22, 2, 4, 5], [7, 81], [4], [24, 1, 3, 85, 18], [67, 37], [4], [11, 4], [42, 2], [73, 6, 2, 22], [38, 14], [40, 2, 3, 94, 5, 64], [24, 31, 1, 3, 19, 4, 1, 3], [46, 55], [5, 4], [28, 14, 4], [16, 8], [21, 1, 15], [32, 10, 54, 57], [3, 2, 3, 4, 56], [18, 87, 53, 3, 19], [1, 4], [12, 7, 4, 5], [6], [43, 24, 26], [1,

In [10]:
# выравнивание текста посредством добавлением нулей в конец предложений (padding='post')
padded = pad_sequences(sequences, maxlen=100, padding='post')
print(padded[0])
print(padded.shape)

[47  5  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0]
(26709, 100)
